<a href="https://colab.research.google.com/github/JBxss/Data-Analysis/blob/main/EvolutionStrategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Evolution Strategies "(μ + λ)-ES"

import numpy as np

def mu_plus_lambda_ES(objective_func, mu, lambda_, num_iterations, num_variables, bounds):
    # Initialize the population
    population = np.random.uniform(bounds[0], bounds[1], size=(mu, num_variables))

    for iteration in range(num_iterations):
        # Evaluate the objective function for each individual in the population
        fitness = np.array([objective_func(individual) for individual in population])

        # Sort the population based on fitness
        sorted_indices = np.argsort(fitness)
        population = population[sorted_indices]

        # Select the top mu individuals as parents for the next generation
        parents = population[:mu]

        # Generate offspring by mutating and recombining the parents
        offspring = np.empty((lambda_, num_variables))
        for i in range(lambda_):
            # Select two random parents' indices for recombination
            parent1_idx, parent2_idx = np.random.choice(range(mu), size=2, replace=False)

            # Perform mutation by adding a small random value to the parent
            parent1 = parents[parent1_idx]
            parent2 = parents[parent2_idx]
            offspring[i] = parent1 + np.random.normal(0, 0.1, size=num_variables)

            # Clip the offspring values to ensure they are within the defined bounds
            offspring[i] = np.clip(offspring[i], bounds[0], bounds[1])

        # Select the top mu offspring based on fitness
        offspring_fitness = np.array([objective_func(individual) for individual in offspring])
        sorted_offspring_indices = np.argsort(offspring_fitness)
        offspring = offspring[sorted_offspring_indices][:mu]

        # Replace the mu least fit individuals in the population with the offspring
        population[-mu:] = offspring

    # Return the best individual and its fitness from the final population
    best_individual = population[0]
    best_fitness = objective_func(best_individual)
    return best_individual, best_fitness


# Example usage
def objective_func(x):
    # Example objective function (minimization)
    return x[0]**2 + x[1]**2

# Set the parameters
mu = 5
lambda_ = 10
num_iterations = 100
num_variables = 2
bounds = (-5, 5)

# Run the (μ + λ)-ES
best_solution, best_fitness = mu_plus_lambda_ES(objective_func, mu, lambda_, num_iterations, num_variables, bounds)

# Print the results
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)


Best solution: [ 0.03921317 -0.03566126]
Best fitness: 0.002809397706114894


In [ ]:
# Evolution Strategies "variant (1+1) - ES"
def evolution_strategy(fitness_function, search_space, num_iterations, sigma):
  """
  Performs an evolution strategy with the "variant (1+1) - ES" algorithm.

  Args:
    fitness_function: A function that takes a point in the search space and returns its fitness.
    search_space: A list of bounds that define the search space.
    num_iterations: The number of iterations to perform.
    sigma: The standard deviation of the Gaussian mutation distribution.

  Returns:
    The best point found after all iterations.
  """

  best_point = None
  best_fitness = float("-inf")

  for i in range(num_iterations):
    # Generate a new point in the search space.
    new_point = search_space + np.random.normal(0, sigma, len(search_space))

    # Evaluate the fitness of the new point.
    new_fitness = fitness_function(new_point)

    # If the new point is better than the best point found so far, replace it.
    if new_fitness > best_fitness:
      best_point = new_point
      best_fitness = new_fitness

  return best_point


# Example of using the evolution strategy to find the minimum of the Rosenbrock function.

def rosenbrock(x):
  return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2


search_space = [-10, 10]
num_iterations = 100
sigma = 1

best_point = evolution_strategy(rosenbrock, search_space, num_iterations, sigma)

print(best_point)

[-12.39422723   9.12188325]


In [ ]:
# Evolution Strategies "Mutation"
def mutation(individual, mutation_strength):
  """
  Performs a mutation on the given individual.

  Args:
    individual: The individual to mutate.
    mutation_strength: The strength of the mutation.

  Returns:
    The mutated individual.
  """

  # Generate a random vector of the same size as the individual.
  random_vector = np.random.normal(0, mutation_strength, individual.shape)

  # Add the random vector to the individual.
  mutated_individual = individual + random_vector

  return mutated_individual


# Example of all of the functional arguments with the implementation.

individual = np.array([1, 2, 3])
mutation_strength = 0.1

mutated_individual = mutation(individual, mutation_strength)

print(mutated_individual)

[0.98461658 1.87051505 2.88976375]


In [ ]:
# Evolution Strategies "Adapting Mutation Step"
import numpy as np

def adapting_mutation_step(population, fitness_function, sigma=1.0, alpha=0.1):
  """
  This function implements the adapting mutation step evolution strategy.

  Args:
    population: A list of individuals.
    fitness_function: A function that takes an individual as input and returns its fitness.
    sigma: The initial mutation step size.
    alpha: The learning rate.

  Returns:
    A new population of individuals.
  """

  # Evaluate the fitness of the current population.
  fitnesses = []
  for individual in population:
    fitnesses.append(fitness_function(individual))

  # Select the best individual.
  best_individual = population[np.argmax(fitnesses)]

  # Generate offspring by mutating the best individual.
  offspring = []
  for i in range(len(population)):
    offspring.append(best_individual + np.random.normal(0, sigma, len(best_individual)))

  # Evaluate the fitness of the offspring.
  offspring_fitnesses = []
  for offspring_individual in offspring:
    offspring_fitnesses.append(fitness_function(offspring_individual))

  # Select the best offspring.
  best_offspring = offspring[np.argmax(offspring_fitnesses)]

  # Update the mutation step size.
  sigma = sigma * (1 + alpha * (best_offspring - best_individual))

  # Return a new population consisting of the best individual and the best offspring.
  return [best_individual, best_offspring]


# Example of all its functional arguments with the implementation:

# Population: A list of individuals.
population = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Fitness function: A function that takes an individual as input and returns its fitness.
def fitness_function(individual):
  return sum(individual)

# Sigma: The initial mutation step size.
sigma = 1.0

# Alpha: The learning rate.
alpha = 0.1

# Run the adapting mutation step evolution strategy.
new_population = adapting_mutation_step(population, fitness_function, sigma, alpha)

# Print the new population.
print(new_population)


[[7, 8, 9], array([ 7.86378886,  7.49786238, 10.33500678])]


In [ ]:
# Evolution Strategies "Recombination"

import random

def recombination(parents, crossover_prob):
  """
  Performs recombination on a list of parents.

  Args:
    parents: A list of parents. Each parent is a list of genes.
    crossover_prob: The probability of crossover.

  Returns:
    A list of offspring. Each offspring is a list of genes.
  """

  offspring = []
  for i in range(len(parents) // 2):
    # Choose two parents randomly.
    parent1 = parents[i]
    parent2 = parents[len(parents) - i - 1]

    # Generate a random number between 0 and 1.
    crossover_point = int(random.random() * len(parent1))

    # Perform crossover if the random number is less than the crossover probability.
    if random.random() < crossover_prob:
      offspring.append(parent1[:crossover_point] + parent2[crossover_point:])
      offspring.append(parent2[:crossover_point] + parent1[crossover_point:])
    else:
      offspring.append(parent1)
      offspring.append(parent2)

  return offspring


# Example of all of the functional arguments with the implementation.

parents = [[1, 2, 3], [4, 5, 6]]
crossover_prob = 0.5

offspring = recombination(parents, crossover_prob)

print(offspring)


[[4, 5, 6], [1, 2, 3]]


In [ ]:
# Evolution Strategies "Selection"
import matplotlib.pyplot as plt

def selection(population, fitness, k=1):
  """
  Selects the best individuals from a population according to their fitness.

  Args:
    population: A list of individuals.
    fitness: A list of fitness values, one for each individual in the population.
    k: The number of individuals to select.

  Returns:
    A list of the k best individuals.
  """

  # Sort the population by fitness.
  population.sort(key=lambda individual: individual['fitness'], reverse=True)

  # Select the top k individuals.
  selected_individuals = population[:k]

  return selected_individuals


# Example of all its functional arguments with the implementation:

population = [
  {"fitness": 1.0},
  {"fitness": 0.5},
  {"fitness": 0.2},
]

fitness = [individual['fitness'] for individual in population]

selected_individuals = selection(population, fitness, k=2)

print(selected_individuals)

# Output:
# [{'fitness': 1.0}, {'fitness': 0.5}]


[{'fitness': 1.0}, {'fitness': 0.5}]
